In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/usi-nlp-practicum-2/imdb_train.csv
/kaggle/input/usi-nlp-practicum-2/winequality-white.csv
/kaggle/input/usi-nlp-practicum-2/winequality-red.csv
/kaggle/input/usi-nlp-practicum-2/imdb_test_data.csv
/kaggle/input/usi-nlp-practicum-2/glove.6B.50d.txt
/kaggle/input/usi-nlp-practicum-2/sample_submission.csv


In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [14]:
import os
print(os.listdir("../input/usi-nlp-practicum-2"))
import warnings
warnings.filterwarnings('ignore')

['imdb_train.csv', 'winequality-white.csv', 'winequality-red.csv', 'imdb_test_data.csv', 'glove.6B.50d.txt', 'sample_submission.csv']


In [15]:
#importing the training data
imdb_data=pd.read_csv('../input/usi-nlp-practicum-2/imdb_train.csv')
print(imdb_data.shape)
print(imdb_data.head(10))
#importing the training data
test_data=pd.read_csv('../input/usi-nlp-practicum-2/imdb_test_data.csv')
print(test_data.shape)
print(test_data.head(10))

(40000, 2)
                                              review sentiment
0  We had STARZ free weekend and I switched on th...  negative
1  I'll admit that this isn't a great film. It pr...  negative
2  I finally found a version of Persuasion that I...  positive
3  The BBC surpassed themselves with the boundari...  positive
4  Much praise has been lavished upon Farscape, b...  negative
5  Of course the plot, script, and, especially ca...  positive
6  This is one of those road movies that would li...  negative
7  What an uninteresting hodge-podge. It could ha...  negative
8  Only a handful of the segments are engaging he...  negative
9  THE GREEN BUTCHERS (Anders Thomas Jensen - Den...  positive
(10000, 2)
   id                                             review
0   1  Not only is this movie a great film for basic ...
1   2  Waitress: Honey, here's them eggs you ordered....
2   3  Many mystery stories follow the standard whodu...
3   4  A space ship cruising through the galaxy encou...


In [16]:
#Summary of the dataset
imdb_data.describe()


,review,sentiment
count,40000,40000
unique,39724,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,20048


In [17]:
#sentiment count
imdb_data['sentiment'].value_counts()

positive    20048
negative    19952
Name: sentiment, dtype: int64

In [18]:
train_reviews=imdb_data.review
train_sentiments=imdb_data.sentiment
train_reviews
test_reviews=test_data.review
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape)


(40000,) (40000,)
(10000,)


In [19]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [20]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)



In [21]:
imdb_data['review'][0]

'We had STARZ free weekend and I switched on the station to see what was on . It was this movie Howling II: The acting was terrible but the eye candy was great. Sybil Danning and Marsha Brown as the afore mentioned eye candy. I was laughing a lot from the few scenes I saw.My friends wonder why I never want to go to Horror movies , If they saw this film they would know why. I would get thrown out for laughing so hard.Just a couple of trivia notes : Reb Brown who played Ben White had played Captain American in a made for TV movie Marsha Brown was Mick Jaggers inspiration for the song "Brown Sugar" Mick has great taste in women for sure.'

In [22]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)
imdb_data['review'][0]

'We had starz free weekend and I switch on the station to see what wa on It wa thi movi howl II the act wa terribl but the eye candi wa great sybil dan and marsha brown as the afor mention eye candi I wa laugh a lot from the few scene I sawmi friend wonder whi I never want to go to horror movi If they saw thi film they would know whi I would get thrown out for laugh so hardjust a coupl of trivia note reb brown who play ben white had play captain american in a made for TV movi marsha brown wa mick jagger inspir for the song brown sugar mick ha great tast in women for sure'

In [23]:
#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)
imdb_data['review'][0]

'starz free weekend switch station see wa wa thi movi howl II act wa terribl eye candi wa great sybil dan marsha brown afor mention eye candi wa laugh lot scene sawmi friend wonder whi never want go horror movi saw thi film would know whi would get thrown laugh hardjust coupl trivia note reb brown play ben white play captain american made TV movi marsha brown wa mick jagger inspir song brown sugar mick ha great tast women sure'

In [24]:
#normalized train reviews
norm_train_reviews=imdb_data.review
norm_train_reviews[0]


'starz free weekend switch station see wa wa thi movi howl II act wa terribl eye candi wa great sybil dan marsha brown afor mention eye candi wa laugh lot scene sawmi friend wonder whi never want go horror movi saw thi film would know whi would get thrown laugh hardjust coupl trivia note reb brown play ben white play captain american made TV movi marsha brown wa mick jagger inspir song brown sugar mick ha great tast women sure'

In [33]:
#Normalized test reviews
norm_test_reviews=test_data.copy()
norm_test_reviews['review']=norm_test_reviews['review'].apply(denoise_text)
norm_test_reviews['review']=norm_test_reviews['review'].apply(simple_stemmer)
norm_test_reviews['review']=norm_test_reviews['review'].apply(remove_stopwords)



In [34]:
norm_test_reviews=norm_test_reviews['review']

In [35]:
norm_test_reviews

0       onli thi movi great film basic cinematographi ...
1       waitress : honey , ' egg ordered. honey , like...
2       mani mysteri stori follow standard whodunit pa...
3       space ship cruis galaxi encount mysteri cargo ...
4       favorit film thi year. great charact plot , di...
                              ...                        
9995    premis may seem goofy , sinc murphy ' charact ...
9996    wife watch thi abort beginning. hate immedi wi...
9997    huge street fighter fan thoroughli enjoy previ...
9998    OK first let say ' still amaz plot sucks , mov...
9999    uhhh ... , even writer ? mayb ' picky , like l...
Name: review, Length: 10000, dtype: object

In [48]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,2))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

In [49]:
imdb_data.head()

,review,sentiment
0,starz free weekend switch station see wa wa th...,negative
1,ill admit thi isnt great film practic scream l...,negative
2,final found version persuas like ann doesnt lo...,positive
3,bbc surpass themselv boundari cross tip velvet...,positive
4,much prais ha lavish upon farscap dont think g...,negative


In [50]:
print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()

BOW_cv_train: (40000, 1940368)
BOW_cv_test: (10000, 1940368)


In [51]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,2))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (40000, 1940368)
Tfidf_test: (10000, 1940368)


In [52]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)


(40000, 1)


In [53]:

#Spliting the sentiment data
train_sentiments=sentiment_data
print(train_sentiments)


#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_reviews,train_sentiments)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(lr_tfidf)

[[0]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]]
LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [54]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_reviews)
print(lr_bow_predict)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

[1 0 1 ... 0 1 0]
[1 0 1 ... 0 1 0]


In [55]:
lr_bow_predicti=lb.inverse_transform(lr_bow_predict)
lr_tfidf_predicti=lb.inverse_transform(lr_tfidf_predict)

In [58]:
test_data['bow']=lr_bow_predicti
test_data['tfid']=lr_tfidf_predicti


In [61]:
test_data['tfid'].value_counts()

positive    6038
negative    3962
Name: tfid, dtype: int64

In [67]:
import os
os.chdir(r'/kaggle/working')
test_data.to_excel('/kaggle/working/Output.xlsx')
test_data
!zip -r file.zip '/kaggle/working/'

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/Output.xlsx (deflated 3%)
  adding: kaggle/working/__notebook_source__.ipynb (deflated 44%)


In [ ]:
import os
os.chdir(r'/kaggle/working')
test_data.to_excel('/kaggle/working/Output.xlsx')
test_data
!zip -r file.zip '/kaggle/working/'
from IPython.display import FileLink
FileLink(r'Output.xlsx')

test_data.to_excel('Output.xlsx')

In [ ]:
test_data.to_excel('/kaggle/working/Output.xlsx')

In [ ]:


#Classification report for bag of words 
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print(lr_bow_report)

In [ ]:
#Classification report for tfidf features
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(cm_tfidf)

#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)
#fitting the svm for bag of words
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
print(svm_bow)
#fitting the svm for tfidf features
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
print(svm_tfidf)

#Predicting the model for bag of words
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

#Accuracy score for bag of words
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)
#Accuracy score for tfidf features
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

#Classification report for bag of words 
svm_bow_report=classification_report(test_sentiments,svm_bow_predict,target_names=['Positive','Negative'])
print(svm_bow_report)
#Classification report for tfidf features
svm_tfidf_report=classification_report(test_sentiments,svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)

#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,svm_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,svm_tfidf_predict,labels=[1,0])
print(cm_tfidf)

#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

#Classification report for bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)


#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
print(cm_tfidf)

#word cloud for positive review words
plt.figure(figsize=(10,10))
positive_text=norm_train_reviews[1]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
positive_words=WC.generate(positive_text)
plt.imshow(positive_words,interpolation='bilinear')
plt.show

#Word cloud for negative review words
plt.figure(figsize=(10,10))
negative_text=norm_train_reviews[8]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
negative_words=WC.generate(negative_text)
plt.imshow(negative_words,interpolation='bilinear')
plt.show